In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [204]:
from region.image import Image
from utils import *
import numpy
import skimage.filters
from skimage.draw import polygon_perimeter, polygon
from scipy.interpolate import interp2d, dfitpack
import scipy.ndimage.morphology

In [150]:
np.set_printoptions(suppress=True)
# numpy.set_printoptions(threshold=numpy.nan)
numpy.set_printoptions(threshold=1000)

In [190]:
def getCorners(image, padding = 0):
    imArr = image.imArr
    return np.array(
        [(0 - padding, 0 - padding), (0 - padding, imArr.shape[0] + padding - 1), (imArr.shape[1] + padding - 1, imArr.shape[0] + padding - 1), (imArr.shape[1] + padding - 1, 0 - padding)])
Image.getCorners = getCorners

# Utils

In [56]:
def getPoints(im, n):
    n = int(n)
    plt.imshow(im.imArr)
    points = plt.ginput(n, timeout = 300, show_clicks=True)
    plt.close()
    print("Points gotten from", im.name, ":", str(points))
    return np.array(points)

def dispPtsOnImg(im, results, tag=False):
    dispPtsOnImgRC(im, results[:, 0], results[:, 1], tag)

def dispPtsOnImgRC(im, rowInds, colInds, tag=False):
    assert len(rowInds) == len(colInds)
    plt.imshow(im)
    plt.plot(rowInds, colInds, 'bo')
    if tag:
        for ind in range(len(rowInds)):
            x, y = rowInds[ind], colInds[ind]
            plt.text(x, y, str(ind), bbox=dict(facecolor='blue', alpha=0.5))
            plt.text(x, y, "%d-%d" % (x,y), bbox=dict(facecolor='red', alpha=0.5))
    plt.waitforbuttonpress()
    plt.close()

In [119]:
def computeH(startPts, endPts):
    assert len(startPts) == len(endPts)
    A, b = createAb(startPts, endPts)
    
    x = numpy.linalg.lstsq(A, b)
    result = np.append(x[0], 1)
    return result.reshape(3, 3)

In [120]:
def createAxxx(startPts, endPts):
    result = np.zeros((len(startPts) * 2 + 1, 9))
    for i in range(len(startPts)):
        startPoint = startPts[i]
        endPoint = endPts[i]
        
        # first row set
        result[i*2, 0] = -1 * startPoint[0]
        result[i*2, 1] = -1 * startPoint[1]
        result[i*2, 2] = -1
        
        result[i*2, 6] = startPoint[0] * endPoint[0]
        result[i*2, 7] = startPoint[1] * endPoint[0]
        result[i*2, 8] = endPoint[0]
        
        
        # second row set
        result[i*2+1, 3] = -1 * startPoint[0]
        result[i*2+1, 4] = -1 * startPoint[1]
        result[i*2+1, 5] = -1
        
        result[i*2+1, 6] = startPoint[0] * endPoint[1]
        result[i*2+1, 7] = startPoint[1] * endPoint[1]
        result[i*2+1, 8] = endPoint[1]
    result[-1, -1] = 1
    
    return result


def createAb(startPts, endPts):
    result = np.zeros((len(startPts) * 2, 8))
    b = np.zeros(len(result))
    for i in range(len(startPts)):
        startPoint = startPts[i]
        endPoint = endPts[i]
        
        # first row set
        result[i*2, 0] = -1 * startPoint[0]
        result[i*2, 1] = -1 * startPoint[1]
        result[i*2, 2] = -1
        
        result[i*2, 6] = startPoint[0] * endPoint[0]
        result[i*2, 7] = startPoint[1] * endPoint[0]
        b[i*2] = -endPoint[0]
        
        
        # second row set
        result[i*2+1, 3] = -1 * startPoint[0]
        result[i*2+1, 4] = -1 * startPoint[1]
        result[i*2+1, 5] = -1
        
        result[i*2+1, 6] = startPoint[0] * endPoint[1]
        result[i*2+1, 7] = startPoint[1] * endPoint[1]
        b[i*2+1] = -endPoint[1]
    # result[-1, -1] = 1
    
    return result, b
    

# def createA(numOfPts):
#     result = np.zeros((numOfPts * 2, 8))
    
#     for i in range()

# def createA(numOfPts):
#     result = np.zeros((numOfPts * 2, 8))
    
#     for i in range()

In [ ]:
dispPtsOnImg(imSproul0.imArr, imSproul0.pts)
dispPtsOnImg(imSproul1.imArr, imSproul1.pts)

In [ ]:
"""
p' = H p
where
    p' = target
    p = source

place on p' the color on p

---

get the points in final image that actually will need a value from source image
    do by calculating the results of the corners of the source img getting transformed,
    then using polygon to get the coordinates that actually lies inside the polygon, so you dont do the coords that isnt in it
    

for every point in p', get the corresponding p with p' * H^-1

"""

In [ ]:
sourcePts = imSproul0.pts
targetPts = imSproul1.pts

H = computeH(sourcePts, targetPts)
print("source:", sourcePts)
print("transformed source:", transf(H, sourcePts))
print("target:", targetPts)

dispPtsOnImg(imSproul0.imArr, sourcePts)
dispPtsOnImg(imSproul1.imArr, transf(H, sourcePts))

In [9]:
def transf(H, res, normalize=True):
    if res.ndim == 1:
        vect = np.append(res, 1)
    else:
        vect = np.vstack((res.T, np.ones(len(res))))
    tfSourceCorners = np.dot(H, vect)
    
    if normalize:
        for i in range(tfSourceCorners.shape[1]):
            corner = tfSourceCorners[:, i]
            normCorner = np.dot(corner, 1/(corner[2]))
            tfSourceCorners[:, i] = normCorner
    return tfSourceCorners[:-1, :].T

In [10]:
def getDimensions(imCornersCoords):
    # assuming imCornersCoords = [[width, height]]
    
    minWidth = int(np.floor(min(imCornersCoords[:, 0])))
    minHeight = int(np.floor(min(imCornersCoords[:, 1])))
    
    maxWidth = int(np.ceil(max(imCornersCoords[:, 0])))
    maxHeight = int(np.ceil(max(imCornersCoords[:, 1])))
    
    totalWidth = maxWidth - minWidth + 1
    totalHeight = maxHeight - minHeight + 1
    
    tup = totalHeight, totalWidth, minHeight, minWidth
    print(tup)
    
    return tup
    

In [112]:
def findAllCoords(tfSourceCorners):
    
    xCorners = tfSourceCorners[:, 0]
    yCorners = tfSourceCorners[:, 1]
    
    flatMinX = int(np.floor(min(xCorners)))
    flatMinY = int(np.floor(min(yCorners)))
    newXCorners = xCorners - flatMinX
    newYCorners = yCorners - flatMinY
    
    
    innerCoordsR, innerCoordsC = polygon(newXCorners, newYCorners)
    finInnerCoordsR, finInnerCoordsC = innerCoordsR + flatMinX, innerCoordsC + flatMinY
    
    
    perimCoordsR, perimCoordsC = polygon_perimeter(xCorners, yCorners)
    allCoordsR = np.append(finInnerCoordsR, perimCoordsR)
    allCoordsC = np.append(finInnerCoordsC, perimCoordsC)

#     allCoordsR = finInnerCoordsR
#     allCoordsC = finInnerCoordsC

#     allCoordsR = perimCoordsR
#     allCoordsC = perimCoordsC

    targetPs = np.vstack((allCoordsR, allCoordsC)).T
    
    return targetPs

In [293]:
class WarpedImageInfo:
    
    def __init__(self, corners, targetPs, rvals, gvals, bvals, iavals):
        self.corners = corners
        self.targetPs = targetPs
        self.rvals = rvals
        self.gvals = gvals
        self.bvals = bvals
        self.iavals = iavals
        
    def createImageObj(self, newImage=None, minH = 0, minW = 0, name = ""):
        if newImage == None:
            finH, finW, minH, minW = getDimensions(self.corners)
            newImage = Image(name, np.zeros((finH, finW, 4)))
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 0] = self.rvals
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 1] = self.gvals
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 2] = self.bvals
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 3] = self.iavals
        newImage.polyCorners = self.corners
        return newImage

def warpTo(imSource, sourcePts, targetPts, padding = 0):

    H = computeH(sourcePts, targetPts)
    sourceCorners = Image.getCorners(imSource, padding)
    tfSourceCorners = transf(H, sourceCorners)

    print("sourceCorners:", sourceCorners)
    print("tfSourceCorners:", tfSourceCorners)

    targetPs = findAllCoords(tfSourceCorners)

    invH = np.linalg.inv(H)

    sourcePs = transf(invH, targetPs)
    
    source_Rchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 0])
    source_Gchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 1])
    source_Bchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 2])
    
    _alpHelp = np.ones(imSource.imArr.shape[:2])
    _alpHelp[_alpHelp.shape[0] // 2, _alpHelp.shape[1] // 2] = 0
    
    _alp2 = scipy.ndimage.morphology.distance_transform_edt(_alpHelp)
    weightedAlph = 1 - (_alp2 / np.amax(_alp2))
#     print(np.amin(weightedAlph), np.amax(weightedAlph))
#     printImage("test.png", weightedAlph)
    
    source_iAchan = interp2d(range(imSource.width), range(imSource.height), weightedAlph)
    
    autoInterp = lambda f, xs, ys: dfitpack.bispeu(f.tck[0], f.tck[1], f.tck[2], f.tck[3], f.tck[4], xs, ys)[0]

    Rvals = autoInterp(source_Rchan, sourcePs[:, 0], sourcePs[:, 1])
    Gvals = autoInterp(source_Gchan, sourcePs[:, 0], sourcePs[:, 1])
    Bvals = autoInterp(source_Bchan, sourcePs[:, 0], sourcePs[:, 1])
    iAvals = autoInterp(source_iAchan, sourcePs[:, 0], sourcePs[:, 1])
    
    return WarpedImageInfo(tfSourceCorners, targetPs, Rvals, Gvals, Bvals, iAvals)

def warpProcess(imBase, imBasePts, imgList, padding = 0):

    warpedResults = []
    for im in imgList:

        imPts = im.pts

        warpedResults.append(warpTo(im, imPts, imBasePts, padding))

    allCorners = np.vstack([warped.corners for warped in warpedResults])
    print(allCorners)
    finH, finW, minH, minW = getDimensions(allCorners)
    # finalImage = Image("finalImage", np.zeros((finH, finW, 3)))

    newWarpedImages = []
    # for warped in warpedResults:
    for i in range(len(warpedResults)):
        warped = warpedResults[i]
        newImage = Image("finalImage", np.zeros((finH, finW, 4)))
        newWarpedImages.append(warped.createImageObj(newImage, minH = minH, minW = minW))
    
    return newWarpedImages

# Alpha Blending

In [307]:
im1 = result[0].imArr
im2 = result[1].imArr


# imMask1 = np.zeros(im1.shape)
# imMask1[im1[:, :, 3] != 0] = 1
# imMask2 = np.zeros(im2.shape)
# imMask2[im2[:, :, 3] != 0] = 1

# printImage("m1.png", im1[:,:,3])
# printImage("m1.png", imMask1)
# printImage("m2.png", imMask2)



def alphaBlendingOutput(imList2):
    assert len(imList2) > 0
    
    imList = [x.imArr for x in imList2]
    
    im1 = imList[0]
    totalAlphas = np.ones(im1.shape[:2])
    totalAlphas = np.zeros(im1.shape[:2])
    for im in imList:
        totalAlphas += im[:,:,3]
    totalAlphas = totalAlphas
#     printImage("ta.png", totalAlphas / np.amax(totalAlphas))
    totalAlphas[totalAlphas == 0] = 1

    finalImage = np.zeros((im1.shape[0], im1.shape[1], 3))
    # div3 = np.dstack((totalAlphas, np.dstack((totalAlphas, totalAlphas))))
    # print(div3.shape)
    # print(finalImage.shape)
    for im in imList:
        weights = np.divide(im[:,:,3], totalAlphas)
        # finalImage[:,:,1] += np.multiply(im[:,:,2], weights)
        weights3 = np.dstack((weights, np.dstack((weights, weights))))
        finalImage += np.multiply(im[:,:,:3], weights3)
    finalImage = np.clip(finalImage, 0, 1)
    return finalImage

finalImage = alphaBlendingOutput(result)

printImage("fin.png", finalImage)


total: [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.00064728 0.00050862 0.        ]
 [0.         0.         0.         ... 0.00075184 0.00061315 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181110230821_ta.png


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181110230829_fin.png


In [306]:
print(np.amin(finalImage))

0.0


# Laplacian blending

In [200]:
def multiResBlendOp(im1, im2, mask, levels, sigma):
    assert im1.shape == im2.shape == mask.shape

    L1 = laplacianPyrOp_3D(im1, levels, sigma)
    L2 = laplacianPyrOp_3D(im2, levels, sigma)
    LM = gaussStackOp_3D(mask, levels, sigma)
    
#     for i in range(len(LM)):
#         printImage(str(i) + ".png", LM[i])
        
#     for i in range(len(L1)):
#         printImage(str(i) + ".png", L1[i])
    LM1 = LM
    LM2 = (1 - LM1)

    L1_post = LM1 * L1
    L2_post = LM2 * L2

    finalL = L1_post + L2_post

    tes = np.zeros(L1[0].shape)

    for i in range(len(L1)):
        tes += finalL[i]

    tes2 = np.clip(tes, -1, 1)

    return tes2

In [123]:
def laplacianPyrOp_3D(im, levels, sigma, scaleB = False):
    gaussStack = gaussStackOp_3D(im, levels, sigma)

    for i in range(levels):
        res = gaussStack[i] - gaussStack[i+1]
        if scaleB:
            finalCurrLayer = (res - res.min()) / (res.max() - res.min())
        else:
            finalCurrLayer = res
        gaussStack[i] = finalCurrLayer

    return gaussStack

In [188]:
def gaussStackOp_3D(im, levels, sigma):
    assert levels > 0
    #inclusive of original img, at layer indexed 0

    result = []
    # newLayer = (lambda: np.zeros(im.shape))

    for i in range(levels+1):
        if i == 0:
            result.append(im)
            continue
        # currLayer = newLayer()
        currLayer = skimage.filters.gaussian(result[i - 1], sigma, mode="constant")
        result.append(currLayer)

    return np.array(result)

In [141]:
imMask = Image.fromPath("../curated_sample_imgs/lower_sproul/mask_test.png", 0.25)
res = multiResBlendOp(imSproul0.imArr, imSproul1.imArr, imMask.imArr, 3, 5)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/filters/_gaussian.py:108: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


In [220]:
im1 = result[0].imArr
im2 = result[1].imArr

def laplacianBlending(im1, im2, level, sig):
    imMask = np.zeros(im1.shape)
    # im1 == 0

    imMask[np.any(im1, 2), 0] = 1
    imMask[np.any(im1, 2), 1] = 1
    imMask[np.any(im1, 2), 2] = 1

    #### cheatsy doodle, the woooooorst way of doing this
    invImMask = 1 - imMask
    invImMask2 = skimage.filters.gaussian(invImMask, sig)
    for i in range(3):
        invImMask2 += skimage.filters.gaussian(invImMask, sig)
    imMask2 = 1 - invImMask2
    imMask2 = np.clip(imMask2, 0, 1)

    res = multiResBlendOp(im1, im2, imMask2, level, sig)

    return res



/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/filters/_gaussian.py:108: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181110210228_test.png


In [142]:
printImage("test.png", laplacianBlending(result[0].imArr, result[1].imArr, 5, 20))

File saved to: output_imgs/20181110184925_test.png


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [226]:
printImage("test.png", laplacianBlending(result2[0].imArr, result2[1].imArr, 5, 20))

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/filters/_gaussian.py:108: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181110213131_test.png


# Scenarios

In [298]:
imSproul0 = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_00.jpg")
imSproul0.readCoordsIn("../curated_sample_imgs/lower_sproul/2_lower_sproul_00.txt")
# imSproul0.pts = imSproul0.pts * 0.25
imSproul1 = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_01.jpg")
imSproul1.readCoordsIn("../curated_sample_imgs/lower_sproul/2_lower_sproul_01.txt")
# imSproul1.pts = imSproul1.pts * 0.25

imgList = [imSproul0, imSproul1]
imBase = imSproul1
imBasePts = imSproul1.pts

result = warpProcess(imBase, imBasePts, imgList, 0)

printImage("fin_lowerSproul.png", alphaBlendingOutput(result))

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """


sourceCorners: [[   0    0]
 [   0 1959]
 [4031 1959]
 [4031    0]]
tfSourceCorners: [[1463.80900135   73.54065945]
 [1515.00328303 1761.78636024]
 [6853.52982941 2466.55774781]
 [7082.44574565 -675.43652028]]
sourceCorners: [[   0    0]
 [   0 1959]
 [4031 1959]
 [4031    0]]
tfSourceCorners: [[   0.    0.]
 [  -0. 1959.]
 [4031. 1959.]
 [4031.    0.]]
[[1463.80900135   73.54065945]
 [1515.00328303 1761.78636024]
 [6853.52982941 2466.55774781]
 [7082.44574565 -675.43652028]
 [   0.            0.        ]
 [  -0.         1959.        ]
 [4031.         1959.        ]
 [4031.            0.        ]]
(3144, 7085, -676, -1)


IndexError: index 3 is out of bounds for axis 2 with size 3

In [308]:
imRoof0 = Image.fromPath("../curated_sample_imgs/roof_esh/roof_esh_00.jpg")
imRoof0.readCoordsIn("../curated_sample_imgs/roof_esh/1_roof_esh_00.txt")
imRoof0.pts = imRoof0.pts
imRoof1 = Image.fromPath("../curated_sample_imgs/roof_esh/roof_esh_01.jpg")
imRoof1.readCoordsIn("../curated_sample_imgs/roof_esh/1_roof_esh_01.txt")
imRoof1.pts = imRoof1.pts

imgList = [imRoof0, imRoof1]
imBase = imRoof1
imBasePts = imRoof1.pts

result2 = warpProcess(imBase, imBasePts, imgList, 0)

printImage("fin_roofEsh.png", alphaBlendingOutput(result2))

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """


sourceCorners: [[   0    0]
 [   0 1959]
 [4031 1959]
 [4031    0]]
tfSourceCorners: [[-2011.61825807  -826.04216801]
 [-1336.05220795  2059.37481888]
 [ 2904.42093564  1790.46658472]
 [ 3411.17764859  -146.82696814]]
sourceCorners: [[   0    0]
 [   0 1959]
 [4031 1959]
 [4031    0]]
tfSourceCorners: [[  -0.   -0.]
 [   0. 1959.]
 [4031. 1959.]
 [4031.   -0.]]
[[-2011.61825807  -826.04216801]
 [-1336.05220795  2059.37481888]
 [ 2904.42093564  1790.46658472]
 [ 3411.17764859  -146.82696814]
 [   -0.            -0.        ]
 [    0.          1959.        ]
 [ 4031.          1959.        ]
 [ 4031.            -0.        ]]
(2888, 6045, -827, -2012)
total: [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.00000914 0.00012162 0.00032154 ... 0.         0.         0.        ]
 [0.00011068 0.00017279 0.00036003 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181110231235_ta.png


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181110231242_fin_roofEsh.png


In [60]:
imRect = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_00.jpg")
imRect.readCoordsIn("../curated_sample_imgs/lower_sproul/rect_orig_lower_sproul_00.txt")
dispPtsOnImg(imRect.imArr, imRect.pts)

targPts = Image.readCoords("../curated_sample_imgs/lower_sproul/rect_targ_lower_sproul_00.txt")
print(targPts)
dispPtsOnImg(imRect.imArr, targPts)

res = warpTo(imRect, imRect.pts, targPts)
im = res.createImageObj()
printImage("rectified.jpg", im.imArr)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[  0.    0. ]
 [  0.  130. ]
 [130.  137.5]
 [137.5   0. ]]


In [311]:

print("Mosaic Roof Esh, Start")
imRoof0 = Image.fromPath("./sample_imgs/roof_esh_00.jpg", 0.25)
imRoof0.readCoordsIn("./sample_imgs/1_roof_esh_00.txt")
imRoof0.pts = imRoof0.pts * 0.25
imRoof1 = Image.fromPath("./sample_imgs/roof_esh_01.jpg", 0.25)
imRoof1.readCoordsIn("./sample_imgs/1_roof_esh_01.txt")
imRoof1.pts = imRoof1.pts * 0.25

imgList = [imRoof0, imRoof1]
imBase = imRoof1
imBasePts = imRoof1.pts

result2 = warpProcess(imBase, imBasePts, imgList, 0)

printImage("alphaBlend_roofEsh.png", alphaBlendingOutput(result2))

im0 = result2[0].imArr[:, :, :3]
im1 = result2[1].imArr[:, :, :3]
printImage("lapBlend_roofEsh.png", laplacianBlending(im0, im1, 5, 20))

print("Mosaic Roof Esh, Done")

Mosaic Roof Esh, Start


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """


sourceCorners: [[   0    0]
 [   0  489]
 [1007  489]
 [1007    0]]
tfSourceCorners: [[-502.90456394 -206.51054183]
 [-334.20571109  514.02083495]
 [ 725.70004217  447.04573259]
 [ 852.13780445  -36.7889837 ]]
sourceCorners: [[   0    0]
 [   0  489]
 [1007  489]
 [1007    0]]
tfSourceCorners: [[   0.    0.]
 [  -0.  489.]
 [1007.  489.]
 [1007.    0.]]
[[-502.90456394 -206.51054183]
 [-334.20571109  514.02083495]
 [ 725.70004217  447.04573259]
 [ 852.13780445  -36.7889837 ]
 [   0.            0.        ]
 [  -0.          489.        ]
 [1007.          489.        ]
 [1007.            0.        ]]
(723, 1512, -207, -503)
total: [[0.00001532 0.00081482 0.00161468 ... 0.         0.         0.        ]
 [0.00021197 0.00078912 0.00153844 ... 0.         0.         0.        ]
 [0.00061785 0.00100811 0.00175804 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.    

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181111000808_ta.png


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181111000809_alphaBlend_roofEsh.png


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/filters/_gaussian.py:108: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


File saved to: output_imgs/20181111000823_lapBlend_roofEsh.png
Mosaic Roof Esh, Done
